### Übung 07 - NUFFT

Die nicht-uniforme diskrete Fouriertransformation lässt sich nicht effizient genug implementieren, sodass sie für die Praxis untauglich ist. Stattdessen interpoliert man die nicht-uniform gesampleten Daten auf ein uniformes Gitter ("Gridding") und wendet dann die FFT an. Das wird "Non-Uniform Fast Fourier Transform" (NUFFT) genannt.


In [9]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
from matplotlib import colors
from helper import *

In [10]:
plt.rcParams["figure.figsize"] = (10,10)

In [11]:
# Platzieren Sie die Datei "phantom_radial_256.npy" im gleichen Ordner, wie dieses Notebook
radial_kspace = np.load("phantom_radial_256.npy")
# Kopieren Sie Ihre Funktion vom letzten Notebook, um die Trajektorie zu generieren oder exportieren Sie die Trajektorie alternativ aus dem letzten Notebook und laden sie hier rein (mit np.save und np.load)
trajectory = compute_spokes(256, 256)

## Aufgabe 1 - Nearest-Neighbor Gridding

Das wohl simpelste Interpolationsverfahren ist, jedem Gitterpunkt einfach den Wert zuzuweisen, der am nächsten dran ist. Implementieren Sie dieses Verfahren. Lassen Sie sich den daraus resultierenden uniformen k-Raum anzeigen.
Tipp: Sie können dafür die Funktion `scipy.interpolate.griddata` verwenden

In [19]:
from scipy import interpolate
print(radial_kspace.shape)
print(trajectory.shape)
flat_traj = np.reshape(trajectory, (256 * 256, 2))
falt_kspace = np.reshape(radial_kspace, 256 * 256)
print(falt_kspace.shape)
print(flat_traj.shape)
inter_k_space = interpolate.griddata(flat_traj, radial_kspace, np.mgrid[-128:127:1, -128:127:1], method='nearest')

(256, 256)
(2, 256, 256)
(65536,)
(65536, 2)


ValueError: different number of values and points

Rekonstruieren Sie nun wie gewohnt mit der ifft das Bild.

Wieso ist diese Art der Interpolation nicht besonders gut geeignet?

<span style="color:blue">(TODO - Schreiben Sie Ihre Antwort hier hin!) </span>


## Aufgabe 2 - NUFFT

Sie werden nun die NUFFT mit einen Kaiser-Bessel-Kernel implementieren. Der Parameter $\beta$ soll wie folgt gewählt sein:

In [ ]:
beta = np.pi * 3.2**0.5

Die Breite des Kernels soll 4 sein:

In [ ]:
kernel_width = 4

Da die Berechnung des Kaiser-Bessel-Fensters für jeden Punkt sehr aufwendig ist, wird meist das Fenster "vorgesamplet". In der eigentlichen Berechnung wird dann einfach das Sample genommen, was am nächsten dran ist.

In [ ]:
kb = np.kaiser(1024*kernel_width, beta)

Da die Fensterfunktion symmetrisch ist, brauchen wir nur eine Hälfte. Außerdem muss die Funktion am Rand 0 sein

In [ ]:
kb_half = kb[int(len(kb)/2):]
kb_half[-1] = 0
plt.plot(np.linspace(0, 0.5, len(kb_half))*kernel_width, kb_half)

Implementieren Sie nun die NUFFT mit dem Kaiser-Bessel-Kernel und visualisieren Sie das Ergebnis

Tipp 1: Denken Sie an die iFFT nach dem Gridding
Tipp 2: Nutzen Sie die gleiche Dichtekompensation für die k-Raum Daten wie im vorherigen Notebook. Ansonsten wird das Bild wieder verwaschen aussehen.

## Aufgabe 3 - Deapodisation

Wenden Sie nun eine Deapodisation an, um das rekonstruierte Bild zu verbessern

Tipp 1: Die iFT des Kaiser-Bessel-Kernels finden Sie in den Folien
Tipp 2: $\frac{1}{I_0(\beta)}$ ist nur ein konstanter Faktor und kann daher weggelassen werden
Tipp 3: Der Wertebereich der Koordinaten im Bildraum ist nach wie vor [-0.5, 0.5]